In [1]:
import collections
import nltk.classify.util, nltk.metrics
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews
from nltk.metrics import scores
from nltk import precision

In [2]:
def evaluate_classifier(featx,collocationFunc):
    #negFiles = movie_reviews.fileids('neg')
    #posFiles = movie_reviews.fileids('pos')
    #negWordsList=[movie_reviews.words(fileids=[f]) for f in negFiles]
    #posWordsList=[movie_reviews.words(fileids=[f]) for f in posFiles]
    #negfeats = [(featx(negWords), 'neg') for negWords in negWordsList]
    #posfeats = [(featx(posWords), 'pos') for posWords in posWordsList]

    negids = movie_reviews.fileids('neg')
    posids = movie_reviews.fileids('pos')
 
    negfeats = [(featx(movie_reviews.words(fileids=[f]),collocationFunc), 'neg') for f in negids]
    posfeats = [(featx(movie_reviews.words(fileids=[f]),collocationFunc), 'pos') for f in posids]

    negcutoff = int(len(negfeats)*3/4)
    poscutoff = int(len(posfeats)*3/4)
 
    trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
    testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
 
    classifier = NaiveBayesClassifier.train(trainfeats)
    refsets = collections.defaultdict(set)
    testsets = collections.defaultdict(set)
 
    for i, (feats, label) in enumerate(testfeats):
            refsets[label].add(i)
            observed = classifier.classify(feats)
            testsets[observed].add(i)
    evaluationMetrics={}
    classifier.show_most_informative_features()
    evaluationMetrics['accuracy']=nltk.classify.util.accuracy(classifier, testfeats)
    evaluationMetrics['posPrec']=nltk.precision(refsets['pos'], testsets['pos'])
    evaluationMetrics['posRecall']=nltk.recall(refsets['pos'], testsets['pos'])
    evaluationMetrics['posF_Score']=nltk.f_measure(refsets['pos'], testsets['pos'])
    evaluationMetrics['negPrec']=nltk.precision(refsets['neg'], testsets['neg'])
    evaluationMetrics['negRecall']=nltk.recall(refsets['neg'], testsets['neg'])
    evaluationMetrics['negF_Score']=nltk.f_measure(refsets['neg'], testsets['neg'])
    return evaluationMetrics

In [3]:
from nltk.corpus import stopwords
stopset = set(stopwords.words('english'))
evaluations=[] 
def stopword_filtered_word_feats(words,collocator):
    return dict([(word, True) for word in words if word not in stopset])
evaluations.append(evaluate_classifier(stopword_filtered_word_feats,None)) 

Most Informative Features
             magnificent = True              pos : neg    =     15.0 : 1.0
             outstanding = True              pos : neg    =     13.6 : 1.0
               insulting = True              neg : pos    =     13.0 : 1.0
              vulnerable = True              pos : neg    =     12.3 : 1.0
               ludicrous = True              neg : pos    =     11.8 : 1.0
                  avoids = True              pos : neg    =     11.7 : 1.0
             uninvolving = True              neg : pos    =     11.7 : 1.0
             fascination = True              pos : neg    =     10.3 : 1.0
              astounding = True              pos : neg    =     10.3 : 1.0
                 idiotic = True              neg : pos    =      9.8 : 1.0


In [4]:
#Bigram Collocations- Handle Cases like “not good”, here B-O-W Approach will Fail
import itertools
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
 
def bigram_word_feats(words, score_fn, n=200):
    bigram_finder = BigramCollocationFinder.from_words(words)
    bigrams = bigram_finder.nbest(score_fn, n)
    return dict([(ngram, True) for ngram in itertools.chain(words, bigrams)])
 
evaluations.append(evaluate_classifier(bigram_word_feats,BigramAssocMeasures.chi_sq))#Works best for this Data
evaluations.append(evaluate_classifier(bigram_word_feats,BigramAssocMeasures.jaccard))
evaluations.append(evaluate_classifier(bigram_word_feats,BigramAssocMeasures.likelihood_ratio))

Most Informative Features
             magnificent = True              pos : neg    =     15.0 : 1.0
             outstanding = True              pos : neg    =     13.6 : 1.0
               insulting = True              neg : pos    =     13.0 : 1.0
       ('matt', 'damon') = True              pos : neg    =     12.3 : 1.0
          ('give', 'us') = True              neg : pos    =     12.3 : 1.0
              vulnerable = True              pos : neg    =     12.3 : 1.0
               ludicrous = True              neg : pos    =     11.8 : 1.0
             uninvolving = True              neg : pos    =     11.7 : 1.0
                  avoids = True              pos : neg    =     11.7 : 1.0
    ('absolutely', 'no') = True              neg : pos    =     10.6 : 1.0
Most Informative Features
             magnificent = True              pos : neg    =     15.0 : 1.0
             outstanding = True              pos : neg    =     13.6 : 1.0
               insulting = True              neg

In [5]:
negFids = movie_reviews.fileids('neg')
posFids = movie_reviews.fileids('pos')

In [6]:
from nltk.collocations import *
from nltk.probability import FreqDist
from nltk.probability import ConditionalFreqDist
word_fd = FreqDist()
label_word_fd = ConditionalFreqDist()

testNegWords = movie_reviews.words(categories=['pos'])
testPosWords = movie_reviews.words(categories=['neg'])

for word in testNegWords:
    word_fd[word.lower()]+=1
    label_word_fd['neg'][word.lower()]+=1
for word in testPosWords:
    word_fd[word.lower()]+=1
    label_word_fd['pos'][word.lower()]+=1
print(word_fd.N(),word_fd.B(),word_fd.most_common(20))
print(label_word_fd.N(),label_word_fd.conditions(),label_word_fd.items())
print(label_word_fd['pos'].N(),label_word_fd['neg'].N())

1583820 39768 [(',', 77717), ('the', 76529), ('.', 65876), ('a', 38106), ('and', 35576), ('of', 34123), ('to', 31937), ("'", 30585), ('is', 25195), ('in', 21822), ('s', 18513), ('"', 17612), ('it', 16107), ('that', 15924), ('-', 15595), (')', 11781), ('(', 11664), ('as', 11378), ('with', 10792), ('for', 9961)]
1583820 ['neg', 'pos'] dict_items([('neg', FreqDist({',': 42448, 'the': 41471, '.': 33714, 'a': 20196, 'and': 19896, 'of': 18636, 'to': 16517, "'": 15268, 'is': 14059, 'in': 11725, ...})), ('pos', FreqDist({',': 35269, 'the': 35058, '.': 32162, 'a': 17910, 'and': 15680, 'of': 15487, 'to': 15420, "'": 15317, 'is': 11136, 'in': 10097, ...}))])
751256 832564


In [7]:
# n_ii = label_word_fd[label][word]
# n_ix = word_fd[word]
# n_xi = label_word_fd[label].N()
# n_xx = label_word_fd.N()
#         w1    ~w1
#      ------ ------
#  w2 | n_ii | n_oi | = n_xi
#      ------ ------
# ~w2 | n_io | n_oo |
#     ------ ------
#      =n_ix         TOTAL = n_xx
# A number of measures are available to score collocations or other associations. The arguments to measure 
# functions are marginals of a contingency table, in the bigram case (n_ii, (n_ix, n_xi), n_xx):
# n_ii = label_word_fd[label][word]
# n_ix = word_fd[word]
# n_xi = label_word_fd[label].N()
# n_xx = label_word_fd.N()
# Chi-Sq Contingency Table : Relating Word w1 with "pos" classification 
#         w1    ~w1
#      ------ ------
# +ve | n_ii | n_oi | = n_xi
#      ------ ------
# -ve | n_io | n_oo |
#     ------ ------
#      =n_ix         TOTAL = n_xx
# n_ix : Total Freq of word w1, n_xi: pos_word_count 
pos_word_count = label_word_fd['pos'].N()
neg_word_count = label_word_fd['neg'].N()
total_word_count = pos_word_count + neg_word_count
 
word_scores = {}

#print(word_fd.items())
for word, freq in word_fd.items():
    pos_score = BigramAssocMeasures.chi_sq(label_word_fd['pos'][word],(freq, pos_word_count), total_word_count)
    neg_score = BigramAssocMeasures.chi_sq(label_word_fd['neg'][word],(freq, neg_word_count), total_word_count)
    word_scores[word] = pos_score + neg_score
import operator
best1 = sorted(word_scores.items(), key=operator.itemgetter(1), reverse=True)[:10000]
bestwords = set([w for w, s in best1])
 
def best_word_feats(words,biGramMeasure):
    return dict([(word, True) for word in words if word in bestwords])
 
evaluations.append(evaluate_classifier(best_word_feats,BigramAssocMeasures.chi_sq))
 
def best_bigram_word_feats(words, score_fn=BigramAssocMeasures.chi_sq, n=200):
    bigram_finder = BigramCollocationFinder.from_words(words)
    bigrams = bigram_finder.nbest(score_fn, n)
    d = dict([(bigram, True) for bigram in bigrams])
    d.update(best_word_feats(words,score_fn))
    return d
evaluations.append(evaluate_classifier(best_bigram_word_feats,BigramAssocMeasures.chi_sq))

Most Informative Features
             magnificent = True              pos : neg    =     15.0 : 1.0
             outstanding = True              pos : neg    =     13.6 : 1.0
               insulting = True              neg : pos    =     13.0 : 1.0
              vulnerable = True              pos : neg    =     12.3 : 1.0
               ludicrous = True              neg : pos    =     11.8 : 1.0
                  avoids = True              pos : neg    =     11.7 : 1.0
             uninvolving = True              neg : pos    =     11.7 : 1.0
             fascination = True              pos : neg    =     10.3 : 1.0
              astounding = True              pos : neg    =     10.3 : 1.0
                 idiotic = True              neg : pos    =      9.8 : 1.0
Most Informative Features
             magnificent = True              pos : neg    =     15.0 : 1.0
             outstanding = True              pos : neg    =     13.6 : 1.0
               insulting = True              neg

In [8]:
for modelEvalMetrics in evaluations:
    print(modelEvalMetrics)

{'posF_Score': 0.7814992025518341, 'negPrec': 0.959349593495935, 'negRecall': 0.472, 'negF_Score': 0.6327077747989276, 'posRecall': 0.98, 'accuracy': 0.726, 'posPrec': 0.649867374005305}
{'posF_Score': 0.8362989323843416, 'negPrec': 0.9202127659574468, 'negRecall': 0.692, 'negF_Score': 0.7899543378995434, 'posRecall': 0.94, 'accuracy': 0.816, 'posPrec': 0.7532051282051282}
{'posF_Score': 0.825, 'negPrec': 0.9, 'negRecall': 0.684, 'negF_Score': 0.7772727272727273, 'posRecall': 0.924, 'accuracy': 0.804, 'posPrec': 0.7451612903225806}
{'posF_Score': 0.8239436619718309, 'negPrec': 0.9120879120879121, 'negRecall': 0.664, 'negF_Score': 0.7685185185185185, 'posRecall': 0.936, 'accuracy': 0.8, 'posPrec': 0.7358490566037735}
{'posF_Score': 0.935114503816794, 'negPrec': 0.9778761061946902, 'negRecall': 0.884, 'negF_Score': 0.9285714285714284, 'posRecall': 0.98, 'accuracy': 0.932, 'posPrec': 0.8941605839416058}
{'posF_Score': 0.9206349206349206, 'negPrec': 0.926829268292683, 'negRecall': 0.912, '

In [74]:
#bigramFinder=BigramCollocationFinder.from_words(movRevWords)
#print(bigramFinder.N)
#help(bigramFinder.apply_freq_filter)
#print(bigramFinder.N)
#bigramFinder.apply_freq_filter(3)
#print("FREQ BASED:",bigramFinder.nbest(bigram_measures.raw_freq,30))
#print("JACCARD:",bigramFinder.nbest(bigram_measures.jaccard,30))
#print("JACCARD:",bigramFinder.nbest(bigram_measures.jaccard,-1))
#bestNGrams=bigramFinder.nbest(bigram_measures.jaccard,-1)
#print("CHI-SQ:",bigramFinder.nbest(bigram_measures.chi_sq,30))
#print("LIKELIHOOD:",bigramFinder.nbest(bigram_measures.likelihood_ratio,30))
#biGfinder.apply_freq_filter()

1764
Help on method apply_freq_filter in module nltk.collocations:

apply_freq_filter(min_freq) method of nltk.collocations.BigramCollocationFinder instance
    Removes candidate ngrams which have frequency less than min_freq.

1764
JACCARD: [('nice', 'hair'), ('/', '10'), ("'", 's'), ('there', 'are'), ('10', ')'), ('television', 'show'), ('we', 'don'), (')', '-'), ("'", 't'), ('t', 'know'), ('based', 'on'), ('they', 'are'), ('this', 'film'), ('film', 'is'), ('don', "'"), ('most', 'of'), (',', 'but'), ('it', "'"), (',', 'and'), ('on', 'a'), ('is', 'not'), ('of', 'course'), ('the', 'film'), ('.', 'it'), ('.', '.'), ('all', 'of'), ("'", 've'), ('doesn', "'"), ('like', 'a'), ('.', 'the'), ('in', 'a'), (',', 'there'), ('that', 'the'), ('it', 'is'), ('of', 'the'), ('for', 'the'), ('is', 'that'), (',', 'it'), ('the', 'movie'), (')', ','), ('.', 'there'), ('on', 'the'), ('by', 'the'), ('the', 'characters'), ('the', 'audience'), ('and', 'the'), (',', 'as'), ('from', 'the'), (',', 'we'), ('with